orders.csv
Contiene todos los pedidos realizados.

order_id (str)
customer_id (str)
order_date (datetime)
order_status (str: Completed, Cancelled, Returned)
shipping_cost (float)
payment_method (str)
total_amount (float)

order_items.csv
Detalle por producto de cada pedido.

order_id (str)
product_id (str)
category (str)
product_name (str)
quantity (int)
unit_price (float)
discount (float, % de descuento aplicado)

In [ ]:
import pandas as pd
import numpy as np

orders = pd.read_csv('orders.csv')
orders_items = pd.read_csv('order_items.csv')
products = pd.read_csv('products.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    249 non-null    object 
 1   category      249 non-null    object 
 2   product_name  249 non-null    object 
 3   cost_price    249 non-null    float64
 4   supplier      249 non-null    object 
dtypes: float64(1), object(4)
memory usage: 9.9+ KB


In [20]:
# Get the total incomes from each product category
relevant_income_columns = orders_items[['order_id', 'product_id', 'category', 'quantity', 'unit_price', 'discount']]
percentage_discount = relevant_income_columns['discount'] / 100
discount_per_item = relevant_income_columns['unit_price'] * percentage_discount
relevant_income_columns['original_income'] = relevant_income_columns['quantity'] * relevant_income_columns['unit_price']
relevant_income_columns['discount_per_unit'] = discount_per_item
relevant_income_columns['total_discount'] = relevant_income_columns['quantity'] * discount_per_item
relevant_income_columns['actual_income'] = relevant_income_columns['original_income'] - relevant_income_columns['total_discount']
relevant_income_columns['product_cost'] = products.set_index('product_id').loc[relevant_income_columns['product_id'], 'cost_price'].values
relevant_income_columns['total_cost'] = relevant_income_columns['quantity'] * relevant_income_columns['product_cost']
relevant_income_columns['shipping_cost'] = orders.set_index('order_id').loc[relevant_income_columns['order_id'], 'shipping_cost'].values
relevant_income_columns

,order_id,product_id,category,quantity,unit_price,discount,original_income,discount_per_unit,total_discount,actual_income,product_cost,total_cost,shipping_cost
0,ORD00001,PROD1238,Books,3,41.94,10,125.82,4.1940,12.5820,113.2380,28.21,84.63,4.00
1,ORD00001,PROD1164,Toys,3,121.19,5,363.57,6.0595,18.1785,345.3915,98.79,296.37,4.00
2,ORD00001,PROD1235,Books,2,118.18,15,236.36,17.7270,35.4540,200.9060,60.72,121.44,4.00
3,ORD00002,PROD1219,Books,2,27.19,15,54.38,4.0785,8.1570,46.2230,22.53,45.06,4.97
4,ORD00003,PROD1073,Clothing,1,70.06,5,70.06,3.5030,3.5030,66.5570,52.80,52.80,14.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3419,ORD01498,PROD1157,Home,1,166.19,10,166.19,16.6190,16.6190,149.5710,83.14,83.14,7.40
3420,ORD01498,PROD1172,Toys,3,121.66,5,364.98,6.0830,18.2490,346.7310,99.68,299.04,7.40
3421,ORD01498,PROD1007,Electronics,3,95.77,0,287.31,0.0000,0.0000,287.3100,56.54,169.62,7.40
3422,ORD01499,PROD1223,Books,1,24.64,5,24.64,1.2320,1.2320,23.4080,17.65,17.65,10.35


In [29]:
relevant_data_for_profit_per_category = relevant_income_columns.groupby('category').agg(
    total_original_income=('original_income', 'sum'),
    total_cost=('total_cost', 'sum'),
    total_discount=('total_discount', 'sum'),
    total_shipping_cost=('shipping_cost', 'sum')
).reset_index()
relevant_data_for_profit_per_category['total_profit'] = (
    relevant_data_for_profit_per_category['total_original_income'] -
    relevant_data_for_profit_per_category['total_cost'] -
    relevant_data_for_profit_per_category['total_discount'] -
    relevant_data_for_profit_per_category['total_shipping_cost']
)
profit_per_category = relevant_data_for_profit_per_category[['category', 'total_profit']].copy()
profit_per_category['total_profit'] = profit_per_category['total_profit'].round(2)
profit_per_category.sort_values(by='total_profit', ascending=False, inplace=True)
profit_per_category

,category,total_profit
1,Clothing,43243.00
3,Home,32600.26
2,Electronics,28598.91
0,Books,27998.94
4,Toys,24597.48
